In [9]:
import torch
from typing import Literal
# they could just copy the pervious token, on another dim?
class MSGPT(torch.nn.Module):
    def __init__(self, backbone: Literal["transformer", "LSTM", "GRU"] = "transformer" ):
        super().__init__()
        if not backbone in ["transformer", "LSTM", "GRU"]:
            raise ValueError("backbone must be one of 'transformer', 'LSTM', 'GRU'")
        self.begin = torch.nn.Embedding(1, 1024)
        self.backbone_type = backbone
        if backbone == "transformer":
            self.pos = torch.nn.Embedding(1024, 1024)
            self.backbone = torch.nn.TransformerEncoder(torch.nn.TransformerEncoderLayer(d_model=1024, nhead=16, batch_first=True), num_layers=8, enable_nested_tensor=False)
        elif backbone == "LSTM":
            self.backbone = torch.nn.LSTM(input_size=1024, hidden_size=1024, num_layers=8, batch_first=True)
        else:
            self.backbone = torch.nn.GRU(input_size=1024, hidden_size=1024, num_layers=8, batch_first=True)
    def forward(self, input):
        # input [B, L, D]
        begin = self.begin(torch.zeros(1, device=input.device, dtype=torch.long)).unsqueeze(0)
        input = torch.cat([begin, input], dim=1)
        if self.backbone_type == "transformer":
            # input = self.pos(torch.arange(input.shape[1], device=input.device)) + input 
            print(input.shape)
            return self.backbone(input)
        else:
            return self.backbone(input)[0]


In [10]:
model = MSGPT("transformer")
model = model.train()
model(torch.randn(1, 10, 1024))

torch.Size([1, 11, 1024])


tensor([[[ 0.9119,  0.2568,  1.9602,  ...,  0.6904,  1.3046, -2.0507],
         [ 0.6826, -1.0355,  0.8275,  ...,  0.7318,  1.3623, -1.6779],
         [ 0.6666, -0.3211,  1.4935,  ...,  0.3075,  1.2610, -0.8673],
         ...,
         [ 1.3572,  0.1298,  1.2824,  ...,  0.6577,  0.5132, -0.8781],
         [ 0.7668, -0.2088,  0.7739,  ...,  0.8059,  1.5599, -1.6975],
         [ 0.0711, -0.6341,  1.4892,  ...,  0.5474,  1.4652, -1.5761]]],
       grad_fn=<NativeLayerNormBackward0>)